In [ ]:
# https://www.tensorflow.org/lite/tutorials/model_maker_image_classification
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *


from time import perf_counter, sleep
import itertools, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

contentPath=os.getcwd()
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
generatorPath

In [ ]:
data = DataLoader.from_folder(generatorPath)
train_ds, val_ds = data.split(0.8)
print()
# print('\nnum_classes:', train_ds.num_classes)
# print('class names', train_ds.index_to_label)

train_size = train_ds.gen_dataset()
item = train_size.cardinality()
print(item)
S = train_size.cardinality() // 8

val_size = val_ds.gen_dataset()
print(val_size.cardinality())
V = val_size.cardinality() // 8
print(S, V)
# print('cardinality: ',genDs.cardinality().numpy(),'\n')
# for item in genDs:
    # print(item)


In [ ]:
bs='''
IMAGE_SIZE = (224,224,3)
BATCH_SIZE = 8

from tensorflow.keras.layers import RandomFlip, RandomTranslation
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_de, val_ds = image_dataset_from_directory(
    directory=generatorPath, shuffle=True, seed=456,
    validation_split=0.2, subset='training', label_mo
)
'''

In [ ]:
def build_dataset(subset):
    return image_dataset_from_directory(
        generatorPath,
        shuffle = False,
        validation_split = 0.2,
        subset = subset,
        label_mode = "categorical",
        seed = 456, # Seed provided when using validation_split 
        # and shuffle  =  True.
        # A fixed seed is used so that the validation set is stable 
        # across runs.
        image_size = (224,224),
        batch_size = 8)

train_ds  =  build_dataset("training")
class_names  =  tuple(train_ds.class_names)
train_size  =  train_ds.cardinality().numpy()
train_ds  =  train_ds.unbatch().batch(BATCH_SIZE)
train_ds  =  train_ds.repeat()

normalization_layer  =  tf.keras.layers.Rescaling(1. / 255)
preprocessing_model  =  tf.keras.Sequential([normalization_layer])
do_data_augmentation  =  True
if do_data_augmentation:
    preprocessing_model.add(tf.keras.layers.RandomRotation(40))
    
    preprocessing_model.add(RandomTranslation(0, 0.05))
    preprocessing_model.add(RandomTranslation(0.05, 0))
    preprocessing_model.add(RandomTranslation(0, 0.1))
    preprocessing_model.add(RandomTranslation(0.1, 0))
    preprocessing_model.add(RandomTranslation(0, 0.2))
    preprocessing_model.add(RandomTranslation(0.2, 0))
    # Like the old tf.keras.preprocessing.image.ImageDataGenerator(),
    # image sizes fixed when reading, then a random zoom is applied.
    # If training inputs larger than image_size, one could also use,
    # RandomCrop with a batch size of 1 and rebatch later.
    
    preprocessing_model.add(tf.keras.layers.RandomZoom(0.2, 0.2))
    preprocessing_model.add(tf.keras.layers.RandomZoom(-0.1, -0.1))
    
    preprocessing_model.add(RandomFlip(mode = "horizontal"))
    preprocessing_model.add(RandomFlip(mode = "vertical"))
    
train_ds  =  train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds  =  build_dataset("validation")
valid_size  =  val_ds.cardinality().numpy()
val_ds  =  val_ds.unbatch().batch(BATCH_SIZE)
val_ds  =  val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))
# STEPS_PER_EPOCH = train_ds.cardinality().numpy()
# VALIDATION_STEPS = val_ds.cardinality(). numpy()
# print(STEPS_PER_EPOCH, VALIDATION_STEPS)
q

In [ ]:
model  =  tflite_model_maker.image_classifier.create(
    train_ds, model_spec = 'mobilenet_v2',
    validation_data = val_ds, # default
    batch_size = None, # default None
    epochs = None,  # default None
    steps_per_epoch = None,  # default None
    train_whole_model = None, # default None
    dropout_rate = None,  # default None
    learning_rate = None,  # default None
    momentum = None, # default None
    shuffle = False,
    use_augmentation = False, # default False
    use_hub_library = True, warmup_steps = None,  # default None
    model_dir = None, # default None
    do_train = True
)

In [ ]:
model.ALLOWED_EXPORT_FORMAT

In [ ]:
loss, accuracy = model.evaluate(data=val_ds)

In [ ]:
model.export(export_dir='.')

In [ ]:
# help(DeprecationWarning)

In [ ]:
testData = DataLoader.from_folder(image_path, shuffle=False)
print('\n', type(testData), '\n')
# help(DataLoader.from_folder)

In [ ]:
train_ds, rest_data = data.split(0.5)
validation_data, val_ds = rest_data.split(0.5)

In [ ]:

from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader
def plotDatasetImages(dataset:ImageClassifierDataLoader):
    """ """
    plt.figure(figsize=(12,12))
    for i, (image, label) in enumerate(dataset.gen_dataset().unbatch().take(25)):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image.numpy(), cmap=plt.cm.gray)
        plt.xlabel(data.index_to_label[label.numpy()])
    plt.show()

plotDatasetImages(dataset=data)

In [ ]:
import tensorflow_hub as hub
module_spec = hub.load_module_spec("path/to/module")
height, width = hub.get_expected_image_size(module_spec)
images = ...  # A batch of images with shape [batch_size, height, width, 3].
module = hub.Module(module_spec)
features = module(images)   # A batch with shape [batch_size, num_features].